In [21]:
import sys
sys.path.append('../utils')

from base import *
from kan import *

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
FILE_NAME = 'us_treasury_rates_large.csv'

In [23]:
df = treasury_data_retrieval(file_name=FILE_NAME)
df.tail()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,
2024-12-02,4.75,4.63,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,4.46,4.36
2024-12-03,4.66,4.56,4.49,4.40,4.27,4.17,4.13,4.11,4.17,4.23,4.50,4.40
2024-12-04,4.65,4.54,4.47,4.38,4.23,4.13,4.09,4.07,4.13,4.19,4.45,4.35
2024-12-05,4.59,4.53,4.46,4.38,4.23,4.15,4.10,4.07,4.12,4.17,4.43,4.33
2024-12-06,4.57,4.50,4.42,4.34,4.19,4.10,4.05,4.03,4.09,4.15,4.42,4.34


In [24]:
df = basis_operations(df)
df.tail()

,Basis_0,Basis_1,Basis_2,Basis_3,Basis_4,Basis_5,Basis_6,Basis_7
Date,,,,,,,,
2024-12-02,4.730691,4.335241,4.337178,4.149289,4.077506,4.137837,4.749234,4.359987
2024-12-03,4.644065,4.355321,4.266334,4.162503,4.101351,4.202328,4.775293,4.400051
2024-12-04,4.629839,4.337400,4.217692,4.124627,4.059989,4.169103,4.714165,4.350041
2024-12-05,4.582256,4.375189,4.186948,4.157661,4.064922,4.126638,4.704006,4.330119
2024-12-06,4.560251,4.320623,4.159402,4.102298,4.015630,4.135312,4.677805,4.340107


In [25]:
test = basis_operations(df, reverse=True)
test.tail()

,Maturity_0,Maturity_1,Maturity_2,Maturity_3,Maturity_4,Maturity_5,Maturity_6,Maturity_7,Maturity_8,Maturity_9,Maturity_10,Maturity_11
Date,,,,,,,,,,,,
2024-12-02,4.730691,4.632349,4.551746,4.397210,4.308409,4.172526,4.102152,4.091315,4.121724,4.191880,4.460011,4.359987
2024-12-03,4.644065,4.571239,4.509532,4.380243,4.274950,4.172658,4.122937,4.121992,4.158615,4.234080,4.499639,4.400051
2024-12-04,4.629839,4.555747,4.492318,4.356006,4.236289,4.132502,4.082834,4.081887,4.119111,4.193726,4.449699,4.350041
2024-12-05,4.582256,4.528626,4.480446,4.365075,4.233787,4.151090,4.096570,4.078784,4.108365,4.175615,4.429267,4.330119
2024-12-06,4.560251,4.498827,4.444856,4.321617,4.194705,4.101254,4.045944,4.039345,4.078434,4.155326,4.419336,4.340107


In [26]:
data, ori_col = full_df_retrieval(df, window_list=[1], lag_list=[1, 2, 3, 4, 5])
all_cols = data.columns

data.tail()

,Basis_0,Basis_1,Basis_2,Basis_3,Basis_4,Basis_5,Basis_6,Basis_7,Basis_0_+_0,Basis_1_+_0,...,Basis_6_-_4_window_1,Basis_7_-_4_window_1,Basis_0_-_5_window_1,Basis_1_-_5_window_1,Basis_2_-_5_window_1,Basis_3_-_5_window_1,Basis_4_-_5_window_1,Basis_5_-_5_window_1,Basis_6_-_5_window_1,Basis_7_-_5_window_1
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-02,4.730691,4.335241,4.337178,4.149289,4.077506,4.137837,4.749234,4.359987,4.730691,4.335241,...,4.750528,4.359615,4.768350,4.381380,4.300640,4.117707,4.056935,4.111388,4.750528,4.359615
2024-12-03,4.644065,4.355321,4.266334,4.162503,4.101351,4.202328,4.775293,4.400051,4.644065,4.355321,...,4.749234,4.359987,4.730691,4.335241,4.337178,4.149289,4.077506,4.137837,4.749234,4.359987
2024-12-04,4.629839,4.337400,4.217692,4.124627,4.059989,4.169103,4.714165,4.350041,4.629839,4.337400,...,4.775293,4.400051,4.644065,4.355321,4.266334,4.162503,4.101351,4.202328,4.775293,4.400051
2024-12-05,4.582256,4.375189,4.186948,4.157661,4.064922,4.126638,4.704006,4.330119,4.582256,4.375189,...,4.714165,4.350041,4.629839,4.337400,4.217692,4.124627,4.059989,4.169103,4.714165,4.350041
2024-12-06,4.560251,4.320623,4.159402,4.102298,4.015630,4.135312,4.677805,4.340107,4.560251,4.320623,...,4.704006,4.330119,4.582256,4.375189,4.186948,4.157661,4.064922,4.126638,4.704006,4.330119


In [27]:
def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5

In [28]:
# Out-of-sample test size, diff between sliding element = test size
test_size = 20
sliding_list = range(270, -30, -30)

# Set variables for cross-validation
truth_df = pd.DataFrame()
naive_df = pd.DataFrame()
kan_df = pd.DataFrame()

# Loop over sliding windows
for sliding in sliding_list:
    
    # Trim original data by sliding window size
    df = data[:len(data)-sliding]

    # Use 2 years of data (500 days) for training
    df_train, df_test = df[-test_size-250:-test_size], df[-test_size:]
    len_train = len(df_train)

    # Append to truth dataframe, if multi-step drop iloc
    truth_df = pd.concat([truth_df, df_test[ori_col].iloc[[-1], :]], axis=0, ignore_index=False)

    # Append to naive dataframe, if multi-step change 1 to test_size
    naive_element = pd.DataFrame([df_train[ori_col].iloc[-1].values] * 1)
    naive_df = pd.concat([naive_df, naive_element], axis=0, ignore_index=True)

    # Initialize predictions array
    pred = list()

    # To predict multi-step use range, to predict only h-ahead-step use equal
    # for h in range(test_size):
    for h in [test_size - 1]:
        # Print checkpoints
        print(f'LAST DAY OF DATASET: {-sliding}, FUTURE STEPS: {h+1}')

        # If h = 0 target columns unchanged
        if h == 0:  
            target_col = ori_col
        # If h > 0 target columns modified
        else:       
            target_col = [f'{element}_+_{h}' for element in ori_col]
        
        # Extract feature columns
        feature_col = [element for element in all_cols if 'window' in element]

        # Cut train data due to direct forecast
        df_train_modified = df_train[:(len_train-h)]

        # Test data is the first row 
        df_test_modified = df_test.iloc[[0]]
        print(len(df_train_modified))

        X_train, y_train = df_train_modified[feature_col], df_train_modified[target_col]
        X_test, y_test = df_test_modified[feature_col], df_test.iloc[h][ori_col]

        X_concat = pd.concat([X_train, X_test], axis=0)

        n_inputs = X_train.shape[1]
        n_outputs = y_train.shape[1]

        dataset = dict()
        dtype = torch.get_default_dtype()
        dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
        dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
        dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
        dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

        # Initialize the model
        model = KAN(width=[n_inputs, 32, n_outputs], grid=5, k=2, seed=42, device=device, symbolic_enabled=False, save_act=False, auto_save=False)

        # Train the model and compute metrics
        results = model.fit(dataset, opt="Adam", lr=0.005, steps=500, metrics=(train_mse, test_mse))

        # loss_fn = loss_fn_eval = lambda x, y: torch.mean((x - y) ** 2)
        # p = 
        # train_loss = loss_fn(p, dataset['train_label'])
        # print(train_loss)
        # torch.sqrt(train_loss).cpu().detach().numpy()


        pred.append(model.forward(dataset['test_input']).cpu().detach().numpy().flatten())
        # print(n_inputs, n_outputs)

    kan_element = pd.DataFrame(pred)
    kan_df = pd.concat([kan_element, kan_df], axis=0, ignore_index=True)


import pickle

dataframes = {
    "naive_df": naive_df,
    "kan_df": kan_df,
    "truth_df": truth_df
}

# Specify the file name
filename = f"dfs_{test_size}_steps_ahead.pkl"

# Pickle the DataFrames into a file
with open(filename, "wb") as file:
    pickle.dump(dataframes, file)

print(f"DataFrames have been saved to {filename}")

LAST DAY OF DATASET: -270, FUTURE STEPS: 20
231


description:   0%|                                                          | 0/500 [00:00<?, ?it/s]

| train_loss: 5.03e-02 | test_loss: 7.11e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:11<00:00, 44.10


LAST DAY OF DATASET: -240, FUTURE STEPS: 20
231


| train_loss: 5.33e-02 | test_loss: 7.66e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:10<00:00, 47.33


...

LAST DAY OF DATASET: -30, FUTURE STEPS: 20
231


| train_loss: 4.22e-02 | test_loss: 1.09e+00 | reg: 0.00e+00 | : 100%|█| 500/500 [00:10<00:00, 47.27


LAST DAY OF DATASET: 0, FUTURE STEPS: 20
231


| train_loss: 3.83e-02 | test_loss: 3.74e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:10<00:00, 47.22

DataFrames have been saved to dfs_20_steps_ahead.pkl


In [30]:
import pickle
import numpy as np 
import pandas as pd 
import math

from sklearn.metrics import mean_squared_error as mse

# cols = truth_df.columns
# mode = 'two-sided'
cross_val_list = [20]

result = dict()
for i in range(8):
    result[i] = list()

for cross_val_window in cross_val_list:
    with open(f"dfs_{cross_val_window}_steps_ahead.pkl", "rb") as file:
        loaded_dataframes = pickle.load(file)

    # Extract individual DataFrames
    naive_df = loaded_dataframes["naive_df"]
    kan_df = loaded_dataframes["kan_df"]
    truth_df = loaded_dataframes["truth_df"]

    for i in range(8):
        y1 = kan_df.iloc[:, i].values
        y2 = naive_df.iloc[:, i].values
        y = truth_df.iloc[:, i].values

        # print(f'{cols[i]}: {dm_test(y, y1, y2, h=1, mode=mode)}')
        # print(f'{cols[i]}, h={cross_val_window}: {mse(y1, y, squared=False):.2f}, {mse(y2, y, squared=False):.2f}')
        if mse(y1, y) < mse(y2, y):
            result[i].append(True)
        else:
            result[i].append(False)

out = pd.DataFrame(result)
out.columns = [f'Basis_{i}' for i in range(8)]
out.index = cross_val_list
out


,Basis_0,Basis_1,Basis_2,Basis_3,Basis_4,Basis_5,Basis_6,Basis_7
20,False,False,False,False,False,True,False,True


In [31]:
kan_df

,0,1,2,3,4,5,6,7
0,5.056624,4.838578,4.243009,4.204908,3.898202,4.265072,4.746259,4.472130
1,4.535608,4.584205,3.758332,3.604705,3.472604,3.453690,4.091483,3.627574
2,5.385783,5.165014,4.547324,4.293717,3.894387,3.839796,4.725731,4.087032
3,5.493140,5.499430,5.015827,4.805905,4.580780,4.307899,5.077660,4.637123
4,5.497186,5.520237,5.219238,5.068422,4.862484,4.542167,5.310967,4.868125
5,5.527145,5.472570,4.880605,4.538012,4.244077,3.979691,4.811885,4.335596
6,5.562175,5.303537,4.625788,4.227295,3.904117,3.821618,4.645080,4.156167
7,5.147082,5.872857,4.501126,4.625140,4.161270,3.737295,4.760108,4.110737
8,5.485382,5.586485,5.304657,4.897720,4.591501,4.002726,5.157231,4.423340
9,5.479118,5.987335,5.638828,5.138017,5.009868,4.134287,5.222639,4.420034


In [33]:
truth_df

,Basis_0,Basis_1,Basis_2,Basis_3,Basis_4,Basis_5,Basis_6,Basis_7
Date,,,,,,,,
2023-11-07,5.547934,5.537941,5.346676,4.861512,4.547162,4.457140,5.295907,4.750719
2023-12-20,5.493810,5.495294,4.741883,4.317909,3.929159,3.603852,4.629847,3.980355
2024-02-05,5.505610,5.399908,4.721860,4.453153,4.171157,3.997618,4.826078,4.350192
2024-03-19,5.504778,5.511803,4.953630,4.671581,4.367010,4.080288,4.886782,4.440149
2024-05-01,5.473537,5.526618,5.124004,4.970940,4.695643,4.420250,5.181448,4.740118
2024-06-13,5.476322,5.528959,4.941748,4.688989,4.286862,4.019531,4.865245,4.400072
2024-07-29,5.526223,5.336852,4.625010,4.364342,4.070538,4.021164,4.908978,4.419527
2024-09-10,5.225100,4.901740,3.784012,3.580271,3.349624,3.765982,4.316365,3.970389
2024-10-23,4.893013,4.512524,4.200108,4.054161,4.006130,4.293233,4.854253,4.510094
